In [1]:
import pandas as pd
import numpy as np

import os
import itertools
import statistics

import json
import re

import scipy
from scipy.stats import ks_2samp

print(pd.__version__)
print(np.__version__)
print(scipy.__version__)


1.3.3
1.20.3
1.7.1


In [2]:
alignment_aa = ['S','R','L','P','T','A',
                'V','G','I','F','Y','C',
                'H','Q','N','K','D','E',
                'M','W','-']

In [3]:
ps6_dir = "./olfr_de/"

filename_cid = pd.read_csv("./cid_info/filename_cid.csv", index_col = 0)
filename_cid = filename_cid.to_dict(orient='index')

for filename in filename_cid:
    filename_cid[filename]['cid'] = str(filename_cid[filename]['cid'])

In [4]:
odor_test_count = {}

for base_name in filename_cid:
    split_name = base_name.split('_')[2:]
    odor_name = split_name[1].split('.')[0]
    if odor_name not in odor_test_count:
        odor_test_count[odor_name] = 0
    odor_test_count[odor_name] += 1
    
multi_conc_tested = {}

for ps6ip_file in os.listdir(ps6_dir):
    conc_odor = ps6ip_file.split('_')[2:]
    conc = conc_odor[0]
    odor_name = conc_odor[1].split('.')[0]
    if odor_test_count[odor_name] > 1:
        ps6ip_file = os.path.join(ps6_dir, ps6ip_file)
        df = pd.read_csv(ps6ip_file, index_col = 0)
        if odor_name not in multi_conc_tested:
            multi_conc_tested[odor_name] = {}
        multi_conc_tested[odor_name][conc] = df

multi_conc_activation = {}

for odor in multi_conc_tested:
    if odor not in multi_conc_activation:
        multi_conc_activation[odor] = {}
    for conc in multi_conc_tested[odor]:
        df = multi_conc_tested[odor][conc]
        sig_or_count = df[(df.logFC > 0) & (df.FDR < 0.05)].shape[0]
        if sig_or_count < 8:
            continue
        multi_conc_activation[odor][conc] = sig_or_count
       
        

In [5]:
tested_resp = {}
sigOR_dict = {}
nonsigOR_dict = {}

for odor in odor_test_count:
    #Pick out concentration for odors tested at multiple concentrations
    if odor_test_count[odor] > 1:
        fewest_or_conc = min(multi_conc_activation[odor], key=multi_conc_activation[odor].get)
        filename = "pS6_DE_"+fewest_or_conc+"_"+odor+".csv"
    else:
    #Rest which are tested at a single concentration
        for base_file in os.listdir(ps6_dir):
            odor_name = base_file.split('_')[3].split('.')[0]
            if odor == odor_name:
                filename = base_file
    file_path = os.path.join(ps6_dir, filename)
    cid = str(filename_cid[filename]['cid'])
    df = pd.read_csv(file_path, index_col = 0)
    df = df.loc[:,['symbol','logFC','FDR']]
    df = df.sort_values(by=['symbol'])
    df = df.reset_index(drop=True)
    if df[(df.logFC > 0) & (df.FDR < 0.05)].shape[0] == 0:
        continue
    if df[(df.logFC > 0) & (df.FDR < 0.05)].shape[0] >= 100:
        continue
    sigORs = df[((df.FDR < 0.05) & (df.logFC > 0))]
    nonsigORs = df[((df.FDR >= 0.25) | (df.logFC < 0))]
    sigOR_dict[cid] = []
    nonsigOR_dict[cid] = []
    sigOR_dict[cid].extend(sigORs['symbol'].values.tolist())
    nonsigOR_dict[cid].extend(nonsigORs['symbol'].values.tolist())

In [6]:
def write_json(data, filepath):
    with open(filepath, 'w') as outfile:
        json.dump(data, outfile)

def load_json(filepath):
    with open(filepath, 'r') as infile:
        data = json.load(infile)
    return data

def load_grantham(granthamfile):
    """
    Input: Grantham scores table, long format
    Output: Grantham scores dictionary
    """
    grantham_dict = {}
    with open(granthamfile, 'r') as f:
        next(f)
        for line in f:
            line = line.strip('\n')
            line = line.split(',')
            if line[0] not in grantham_dict:
                grantham_dict[line[0]]={}
            grantham_dict[line[0]][line[1]] = int(line[2])
    return grantham_dict

def read_fasta(fastafile):
    """
    Input: FASTA file of aligned Olfr (gaps OK)
           entries must start with '>' & sequence must be immediately next
           no other lines allowed
    Output: dictionary with keys equal identity of entry
            values are aa sequence associated with entry 
    """
    fasta_dict = {}
    header = ''
    sequence = ''
    with open(fastafile, 'r') as f:
        for line in f:
            if '>' in line:
                if sequence != '':
                    fasta_dict[header] = sequence
                header = line[1:].strip('\n')
                sequence = ''
            else: 
                sequence += line.strip('\n')
        fasta_dict[header] = sequence
    return fasta_dict

def alignment_info(fasta_dict):
    """
    Input: FASTA dictionary with gaps
    Output: FASTA dictionary with original amino acid + position
    """
    alignment_info = {}
    for entry in fasta_dict:
        alignment_info[entry] = {}
        counter = 0
        for position, aa in enumerate(fasta_dict[entry]):
            if aa != '-':
                counter += 1
                alignment_info[entry][position+1] = aa+'_'+str(counter)
            else:
                alignment_info[entry][position+1] = aa
    return alignment_info

def remove_gaps(fasta_dict):
    """
    Input: FASTA dictionary with gaps
    Output: FASTA dictionary with gaps removed
    """
    no_gaps = {}
    for entry in fasta_dict:
        seq = fasta_dict[entry]
        seq = re.sub('-','',seq)
        no_gaps[entry] = seq
    return no_gaps

def write_fasta(fasta_dict, file_path):
    """
    Input: FASTA dictionary, path to write to
    Output: FASTA file
    """
    with open(file_path, "w") as outfile:
        for key in fasta_dict:
            outfile.write('{0}\n{1}\n'.format(">"+key, fasta_dict[key]))

def gap_calculator(fastafile):
    """
    Input: FASTA file
    Output: Percentage of gaps at each position
    """
    percent_gap = {}
    for position in fastafile:
        aa_count = 0
        gap_count = 0
        for residue in fastafile[position]:
            if residue == '-':
                gap_count += fastafile[position][residue]
            else:
                aa_count += fastafile[position][residue]
        percent_gap[position] = (gap_count/(gap_count+aa_count))*100
    return percent_gap
                        
def aa_composition(aligned_fasta):
    """
    Input: Aligned FASTA with gaps
    Output: Amino acid composition at all positions
    """
    consus = {}
    for key in aligned_fasta:
        for position, aa in enumerate(aligned_fasta[key]):
            consus[position] = {}
    for position in consus:
        for aa_possibility in alignment_aa:
            consus[position][aa_possibility] = 0
    for key in aligned_fasta:
        for position, aa in enumerate(aligned_fasta[key]):
            consus[position][aa] += 1
    return consus

def length_calculator(percent_gaps):
    """
    Input: Percent gaps per position
    Output: How sequence length changes as a function of percent gaps
    """
    seq_length = {}
    align_length = len(percent_gaps)
    for gap_occupancy in range(101):
        counter = 0
        for position in percent_gaps:
            if percent_gaps[position] > gap_occupancy :
                counter += 1
        seq_length[100-gap_occupancy] = align_length - counter
    return seq_length

def full_distances(aligned_fasta, grantham_vals, avg_grantham):
    """
    Input: Aligned FASTA file with gaps & grantham scores
    Output: Full grantham distance between entries
    """
    distances = {}
    for key1 in aligned_fasta:
        distances[key1] = {}
        for key2 in aligned_fasta:
            distances[key1][key2] = 0
    for key1 in distances:
        for key2 in distances[key1]:
            if key2 == key1:
                continue
            for position, residue in enumerate(aligned_fasta[key1]):
                aa1 = aligned_fasta[key1][position]
                aa2 = aligned_fasta[key2][position]
                if aa1 == '-' and aa2 == '-':
                    distances[key1][key2] += 0
                elif aa1 != '-' and aa2 != '-':
                    distances[key1][key2] += grantham_vals[aa1][aa2]
                else:
                    #impute average Grantham for this position
                    distances[key1][key2] += float(avg_grantham[str(position)]) 
    return distances

def average_grantham(aligned_fasta, grantham_vals):
    """
    Input: Aligned FASTA file with gaps & grantham scores
    Output: Calculates average pairwise grantham score at
    each position.
    """
    consus = {}
    for key in aligned_fasta:
        for position, aa in enumerate(aligned_fasta[key]):
            if position not in consus:
                consus[position] = []
            consus[position].append(aa)        
    avg_consus = {}
    for position in consus:
        length = 0
        sum_grantham = 0
        aa_list = consus[position]
        for aa1, aa2 in itertools.combinations(aa_list, 2):
            if aa1 != '-' and aa2 != '-': 
                length += 1
                sum_grantham += grantham_vals[aa1][aa2]
        if length == 0:
            length = 1
        avg_consus[position] = sum_grantham/length
    return avg_consus

def get_seqs(sig_or_dict, aligned_fasta):
    """
    Input: Dictionary of receptors significant responding to some odors
    Output: Sequences of significantly activated receptors if they are in the alignment file
    """
    sig_seq = {}
    for odor in sig_or_dict:
        sig_seq[odor] = {}
        for sig_rec in sig_or_dict[odor]:
            if sig_rec in aligned_fasta:
                sig_seq[odor][sig_rec] = aligned_fasta[sig_rec]
    return sig_seq

def get_grantham_dist(sig_sequences, grantham_vals, avg_grantham):
    """
    Input: Sequences dictionary, grantham dictionary, avg grantham scores
    Output: Generates a dictionary for the grantham score distribution at each position
    """
    sig_grantham_distance = {}
    for odor in sig_sequences:
        sig_grantham_distance[odor] = {}
        for rec1, rec2 in itertools.combinations(sig_sequences[odor], 2):
            for position, aa in enumerate(sig_sequences[odor][rec1]):
                aa1 = sig_sequences[odor][rec1][position]
                aa2 = sig_sequences[odor][rec2][position]
                if position not in sig_grantham_distance[odor]:
                    sig_grantham_distance[odor][position] = []
                if aa1 != '-' and aa2 != '-':
                    sig_grantham_distance[odor][position].append(grantham_vals[aa1][aa2])
                elif aa1 == '-' and aa2 == '-':
                    sig_grantham_distance[odor][position].append(0)
                else:
                    sig_grantham_distance[odor][position].append(avg_grantham[str(position)])
    return sig_grantham_distance

def Average(lst):
    """
    Input:list
    Output: Average value of list
    """
    return sum(lst) / len(lst)

def delta_length_calc(gap_length_estimate, increment):
    """
    Input: Gap length estimate dictionary & increment check
    Output: How much the length changes as a function of amino
    acid occupancy
    """
    delta_length = {}
    for gap_occ in range(0,100,increment):
        current_length = gap_length_estimate[gap_occ]
        new_length = gap_length_estimate[gap_occ+increment]
        length_change = current_length-new_length
        delta_length[gap_occ+increment] = length_change
    return delta_length

def gaps_removed(aligned_fasta, percent_gaps, gap_occupancy_cutoff):
    """
    Input: Aligned FASTA file. Percent gaps per position. Cutoff for gaps per position.
    Output: New FASTA file with positions with more gaps than the cutoff removed.
    """
    marked_fasta_dict = {}
    for olfr in aligned_fasta:
        orig_seq = list(aligned_fasta[olfr])
        for position,aa in enumerate(aligned_fasta[olfr]):
            if percent_gaps[position] > gap_occupancy_cutoff:
                orig_seq[position] = '*'
        new_seq = "".join(orig_seq)
        marked_fasta_dict[olfr] = new_seq
    new_fasta_dict = {}
    for olfr in marked_fasta_dict:
        orig_seq = marked_fasta_dict[olfr]
        new_seq = re.sub('[*]','',orig_seq)
        new_fasta_dict[olfr] = new_seq
    return new_fasta_dict

   
        


In [7]:
def extract_mod_info(aligned_fasta, marked_fasta_dict, gap_occupancy_cutoff):
    """
    Input: Original FASTA alignment dictionary & modified alignment with gaps removed.
    The gap occupancy cutoff value should be the same as fxn(gaps_removed).
    Output: Dictionary with positions of original protein & modified sequence.
    """
    marked_fasta_dict = {}
    for olfr in aligned_fasta:
        orig_seq = list(aligned_fasta[olfr])
        for position,aa in enumerate(aligned_fasta[olfr]):
            if percent_gaps[position] > gap_occupancy_cutoff:
                orig_seq[position] = '*'
        new_seq = "".join(orig_seq)
        marked_fasta_dict[olfr] = new_seq
    refined_dict = {}
    for olfr in aligned_fasta:
        refined_dict[olfr] = {}
        prot_position = 0
        mod_position = 0
        for position,aa in enumerate(aligned_fasta[olfr]):
            if aligned_fasta[olfr][position] != '-':
                prot_position += 1
            if marked_fasta_dict[olfr][position] != '*' and aligned_fasta[olfr][position] != '-':
                mod_position += 1
            if marked_fasta_dict[olfr][position] != '*' and aligned_fasta[olfr][position] == '-':
                mod_position += 1
            #write data
            if aa == '-' and marked_fasta_dict[olfr][position] != '-':
                refined_dict[olfr][position+1] = "-"
            elif aa == '-' and marked_fasta_dict[olfr][position] == '-':
                refined_dict[olfr][position+1] = "prot_position: GAP, mod_position: GAP"
            elif aa != '-' and marked_fasta_dict[olfr][position] == '*':
                refined_dict[olfr][position+1] = "prot_position: "+str(prot_position)+","+" mod_position: REMOVED"
            else:
                refined_dict[olfr][position+1] = "prot_position: "+str(prot_position)+","+" mod_position: "+str(mod_position)
    return refined_dict
            
def extract_info(fasta_60p_info, info_type):
    """
    Input: FASTA info dictionary.
    Info type = 0 retrieves position in original aa sequence.
    Info type = 1 retrieves position in attenuated aa sequence after removing low aa positions.
    Output: Dictionary with position info.
    """
    prot_position = {}
    for olfr in fasta_60p_info:
        prot_position[olfr] = {}
        for position in fasta_60p_info[olfr]:
            info = fasta_60p_info[olfr][position]
            if info == '-':
                prot_position[olfr][position] = "-"
            else:
                prot_position[olfr][position] = fasta_60p_info[olfr][position].split(',')[info_type].split(':')[1].strip(' ')
    return prot_position


         

In [8]:
def find_nearest_nr(input_odor, input_receptor, input_distance):
    """
    Dependent data structure: 
    Input: For a given odor, receptor, and it's distance to another receptor...
    Output: Find the nearest nonresponding receptor based on full length distnaces.
    Output is a tuple of 2:
        [0] = Nearest non-responding receptor
        [1] = Nearest non-responding receptor distance to input receptor
    """
    nearest_nr = min({k:v for k,v in olfr_distances_60p_aaIdentity[input_receptor].items()
                     if k in non_sig_sequences_60p_aaIdentity[input_odor].keys()}.items(),
                    key=lambda e: abs(input_distance-e[1]))
    return nearest_nr

#Odors 10925, 8094, 440917 have only 1 responding receptor
#Pairwise analysis of receptors cannot be done for these odors
def make_nearest_nr_dict(input_dict):
    """
    Dependent data structure: olfr_distances_60p_aaIdentity
    Dependent function: find_nearest_nr
    Input: sig_sequences_60p_aaIdentity
    Output: Dictionary with the following structure
        [0] = [odor]
        [1] = [responding receptor 1, comparing from]
        [2] = [responding receptor 2, comparing to]
        [3] = [(nearest non-responding receptor, distance to nearest non-responder)]
              based on distance between responding receptor 1 & responding receptor 2
    """
    nearest_nr_dict = {}
    for odor in input_dict:
        nearest_nr_dict[odor] = {}
        for olfr1, olfr2 in itertools.combinations(input_dict[odor].keys(), 2):
            distance_val = olfr_distances_60p_aaIdentity[olfr1][olfr2]
            nearest_nr_olfr1 = find_nearest_nr(odor, olfr1, distance_val)
            nearest_nr_olfr2 = find_nearest_nr(odor, olfr2, distance_val)
            if olfr1 not in nearest_nr_dict[odor]:
                nearest_nr_dict[odor][olfr1] = {}
            if olfr2 not in nearest_nr_dict[odor]:
                nearest_nr_dict[odor][olfr2] = {}
            nearest_nr_dict[odor][olfr1][olfr2] = nearest_nr_olfr1[0], nearest_nr_olfr1[1]
            nearest_nr_dict[odor][olfr2][olfr1] = nearest_nr_olfr2[0], nearest_nr_olfr2[1]
    return nearest_nr_dict

def convergent_distances(input_dict):
    """
    Input: nearest_nr_dict_60p_aaIdentity
    Output: Dictionary for list of grantham distances at each position between amino
    acids of responding receptors and nearest non-responder based on full length sequence
    distnace.
    """
    conv_evolve_dist_60p_aaIdentity = {}
    for odor in input_dict:
        conv_evolve_dist_60p_aaIdentity[odor] = {}
        for resp1 in input_dict[odor]:
            resp1_seq = fasta_60p_aaIdentity[resp1]
            for resp2 in input_dict[odor][resp1]:
                nonresp = input_dict[odor][resp1][resp2][0]
                nonresp_seq = fasta_60p_aaIdentity[nonresp]
                for position, resp_aa in enumerate(resp1_seq):
                    nonresp_aa = nonresp_seq[position]
                    if position not in conv_evolve_dist_60p_aaIdentity[odor]:
                        conv_evolve_dist_60p_aaIdentity[odor][position] = []
                    if resp_aa == '-' and nonresp_aa == '-':
                        gd = 0
                    elif resp_aa != '-' and nonresp_aa != '-':
                        gd = grantham_vals[resp_aa][nonresp_aa]
                    else:
                        gd = avg_grantham_60p_aaIdentity[str(position)]
                    conv_evolve_dist_60p_aaIdentity[odor][position].append(gd)
    return conv_evolve_dist_60p_aaIdentity
                
def ks_test(dict1, dict2):
    ks_out = {}
    for odor in dict1:
        if len(dict1[odor]) == 0:
            continue
        ks_out[odor] = {}
        for position in dict1[odor]:
            lst1 = dict1[odor][position]
            lst2 = dict2[odor][position]
            ks_results = ks_2samp(lst1, lst2)
            ks_out[odor][position] = ks_results[0], ks_results[1]
    return ks_out               
            
            

In [9]:
aligned_fasta = read_fasta("./mouseOR_alignment/mouseOR_alignment.fasta")
grantham_vals = load_grantham("./mouseOR_alignment/grantham_table.csv")
fasta_composition = aa_composition(aligned_fasta)
percent_gaps = gap_calculator(fasta_composition)
fasta_info = alignment_info(aligned_fasta)
gap_length_estimate = length_calculator(percent_gaps)
delta_length = delta_length_calc(gap_length_estimate, 5)
#avg_grantham = average_grantham(aligned_fasta, grantham_vals)
#write_json(avg_grantham, "./mouseOR_alignment/avg_grantham.json")
avg_grantham = load_json("./mouseOR_alignment/avg_grantham.json")
#olfr_distances = full_distances(aligned_fasta, grantham_vals, avg_grantham)
#olfr_distances = pd.DataFrame.from_dict(olfr_distances)
#olfr_distances.to_csv("./mouseOR_alignment/OR_distances.csv")
olfr_distances = pd.read_csv("./mouseOR_alignment/OR_distances.csv", index_col = 0).to_dict()
sig_sequences = get_seqs(sigOR_dict, aligned_fasta)
non_sig_sequences = get_seqs(nonsigOR_dict, aligned_fasta)

In [10]:
fasta_60p_aaIdentity = gaps_removed(aligned_fasta, percent_gaps, 40)
sig_sequences_60p_aaIdentity = get_seqs(sigOR_dict, fasta_60p_aaIdentity)
non_sig_sequences_60p_aaIdentity = get_seqs(nonsigOR_dict, fasta_60p_aaIdentity)
fasta_60p_info = extract_mod_info(aligned_fasta, fasta_60p_aaIdentity, 40)
orig_prot_position = extract_info(fasta_60p_info, 0)
mod_prot_position = extract_info(fasta_60p_info, 1)
fasta_composition_60p_aaIdentity = aa_composition(fasta_60p_aaIdentity)
#avg_grantham_60p_aaIdentity = average_grantham(fasta_60p_aaIdentity, grantham_vals)
#write_json(avg_grantham_60p_aaIdentity, "./mouseOR_alignment/avg_grantham_60p_aaIdentity.json")
avg_grantham_60p_aaIdentity = load_json("./mouseOR_alignment/avg_grantham_60p_aaIdentity.json")
#olfr_distances_60p_aaIdentity = full_distances(fasta_60p_aaIdentity, grantham_vals, avg_grantham_60p_aaIdentity)
#write_json(olfr_distances_60p_aaIdentity, "./mouseOR_alignment/olfr_distances_60p_aaIdentity.json")
olfr_distances_60p_aaIdentity = load_json("./mouseOR_alignment/olfr_distances_60p_aaIdentity.json")
#pd.DataFrame.from_dict(olfr_distances_60p_aaIdentity).to_csv("./mouseOR_alignment/olfr_distances_60p_aaIdentity.csv")
nearest_nr_dict_60p_aaIdentity = make_nearest_nr_dict(sig_sequences_60p_aaIdentity)
#Compute pairwise distances between responding receptors for every position in every odor
sig_grantham_dist_60p_aaIdentity = get_grantham_dist(sig_sequences_60p_aaIdentity, grantham_vals, avg_grantham_60p_aaIdentity)
#Compute pairwise distances between responding receptors & nearest non-responders
#for every position in every odor
conv_nr_distances_60p_aaIdentity = convergent_distances(nearest_nr_dict_60p_aaIdentity)
#Kolmogorov-smirnov statistical test
ks_60p_aaIdentity = ks_test(sig_grantham_dist_60p_aaIdentity, conv_nr_distances_60p_aaIdentity)



In [11]:
ks_60p_pval = {}

for odor in ks_60p_aaIdentity:
    ks_60p_pval[odor] = {}
    for position in ks_60p_aaIdentity[odor]:
        ks_60p_pval[odor][position] = ks_60p_aaIdentity[odor][position][1]


ks_count = {}

for odor in ks_60p_aaIdentity:
    for position in ks_60p_aaIdentity[odor]:
        if position not in ks_count:
            ks_count[position] = 0
        if ks_60p_aaIdentity[odor][position][1] < 0.05:
            ks_count[position] += 1

ks_df = pd.DataFrame.from_dict(ks_count, orient = 'Index').sort_values(by=0, ascending = False)

ks_pval_df = pd.DataFrame.from_dict(ks_60p_pval)
ks_pval_df = ks_pval_df.reset_index().melt('index')
ks_pval_df.columns = ['position','odor_cid','pVal']
#ks_pval_df.to_csv("./mouseOR_alignment/ks_pval.csv")

In [16]:
#with open("./mouseOR_alignment/mouseOR_60p_aaIdentity.fasta",'w') as f:
#    for olfr in fasta_60p_aaIdentity:
#        f.write('>{0}\n{1}\n'.format(olfr, fasta_60p_aaIdentity[olfr]))